In [3]:
import torch
import torch.nn as nn
import torch.autograd as autograd
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
import pandas as pd
import numpy as np

In [4]:
# Compute derivatives for velocities and accelerations
def compute_derivatives(data, dt):
    velocities = np.gradient(data, dt, axis=0)
    accelerations = np.gradient(velocities, dt, axis=0)
    return velocities, accelerations

# Load generalized coordinates from preprocessed dataset
def load_generalized_coordinates(data, dt):
    q = torch.tensor(data, dtype=torch.float32, requires_grad=True)

    # Compute velocities (q_dot) and accelerations (q_ddot)
    q_dot_np, q_ddot_np = compute_derivatives(data, dt)
    q_dot = torch.tensor(q_dot_np, dtype=torch.float32)
    q_ddot = torch.tensor(q_ddot_np, dtype=torch.float32)

    return q, q_dot, q_ddot

In [5]:
file_path = "./humanoid.csv"  # File created in earlier processing step
data = pd.read_csv(file_path)

In [6]:
columns = [x for x in data.columns if "joint" in x and "Finger" in x]

In [7]:
dt = np.round(data.timestep.diff()[1], 2)
dt

0.01

In [8]:
q, q_dot, q_ddot = load_generalized_coordinates(data[columns].values, dt)


In [9]:
# Neural network for the mass matrix H(q; \psi)
class MassMatrixNN(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super(MassMatrixNN, self).__init__()
        self.network = nn.Sequential(
            nn.Linear(input_dim, hidden_dim),
            nn.Tanh(),
            nn.Linear(hidden_dim, hidden_dim),
            nn.Tanh(),
            nn.Linear(hidden_dim, output_dim * (output_dim + 1) // 2)  # Output lower triangular matrix
        )
        self.epsilon = 1e-3  # Small positive offset for stability
        self.alpha = 1.0     # Shift parameter for diagonal elements

    def forward(self, q):
        L_elements = self.network(q)  # Predict lower triangular elements
        batch_size = L_elements.size(0)
        output_dim = int((L_elements.size(1) * 2) ** 0.5)  # Derive matrix size

        # Create lower triangular matrix with diagonal offsets
        L = torch.zeros(batch_size, output_dim, output_dim, device=q.device)
        tril_indices = torch.tril_indices(row=output_dim, col=output_dim, offset=0)
        L[:, tril_indices[0], tril_indices[1]] = L_elements

        # Apply the diagonal shift for positive definiteness
        diag_indices = torch.arange(output_dim, device=q.device)
        L[:, diag_indices, diag_indices] += self.alpha
        L[:, diag_indices, diag_indices] = torch.nn.functional.softplus(L[:, diag_indices, diag_indices]) + self.epsilon

        # Compute positive definite mass matrix H = L L^T
        H = torch.bmm(L, L.transpose(-1, -2))
        return H

# Neural network for the potential energy V(q; \phi)
class PotentialEnergyNN(nn.Module):
    def __init__(self, input_dim, hidden_dim):
        super(PotentialEnergyNN, self).__init__()
        self.network = nn.Sequential(
            nn.Linear(input_dim, hidden_dim),
            nn.Tanh(),
            nn.Linear(hidden_dim, hidden_dim),
            nn.Tanh(),
            nn.Linear(hidden_dim, 1)  # Output scalar potential energy
        )

    def forward(self, q):
        V = self.network(q)
        return V

# Forward dynamics model: \ddot{q} = f(q, \dot{q}, \tau; \psi, \phi)
class ForwardDynamics(nn.Module):
    def __init__(self, mass_matrix_nn, potential_energy_nn):
        super(ForwardDynamics, self).__init__()
        self.mass_matrix_nn = mass_matrix_nn
        self.potential_energy_nn = potential_energy_nn

    def forward(self, q, q_dot, tau):
        H = self.mass_matrix_nn(q)  # Mass matrix
        V = self.potential_energy_nn(q)  # Potential energy

        # Compute gradients using autograd
        V_grad = autograd.grad(V.sum(), q, create_graph=True)[0]

        H_grad = torch.stack([autograd.grad(H[i].sum(), q, create_graph=True, retain_graph=True)[0] for i in range(H.size(0))])
        H_dot_q = torch.einsum("bij,bj->bi", H_grad, q_dot)

        q_dot_H_grad_q_dot = torch.einsum("bi,bj,bij->b", q_dot, q_dot, H_grad) / 2.0

        # Compute \ddot{q}
        H_inv = torch.linalg.inv(H)
        q_ddot = torch.bmm(H_inv, (tau - H_dot_q - q_dot_H_grad_q_dot.unsqueeze(-1) - V_grad.unsqueeze(-1))).squeeze(-1)
        return q_ddot

# Loss functions
class DynamicsLoss(nn.Module):
    def __init__(self):
        super(DynamicsLoss, self).__init__()

    def forward(self, q, q_dot, q_ddot, tau, mass_matrix_nn, potential_energy_nn):
        # Mass matrix and potential energy
        H = mass_matrix_nn(q)
        V = potential_energy_nn(q)

        # Compute gradients
        V_grad = autograd.grad(V.sum(), q, create_graph=True)[0]

        H_grad = torch.stack([autograd.grad(H[i].sum(), q, create_graph=True, retain_graph=True)[0] for i in range(H.size(0))])
        H_dot_q = torch.einsum("bij,bj->bi", H_grad, q_dot)

        q_dot_H_grad_q_dot = torch.einsum("bi,bj,bij->b", q_dot, q_dot, H_grad) / 2.0

        # Compute forward dynamics prediction
        H_inv = torch.linalg.inv(H)
        q_ddot_pred = torch.bmm(H_inv, (tau - H_dot_q - q_dot_H_grad_q_dot.unsqueeze(-1) - V_grad.unsqueeze(-1))).squeeze(-1)

        # Forward dynamics loss
        forward_loss = torch.mean((q_ddot - q_ddot_pred) ** 2)

        # Compute inverse dynamics prediction for torque
        tau_pred = torch.bmm(H, q_ddot.unsqueeze(-1)).squeeze(-1) + H_dot_q + q_dot_H_grad_q_dot.unsqueeze(-1).squeeze(-1) + V_grad

        # Inverse dynamics loss
        inverse_loss = torch.mean((tau - tau_pred) ** 2)

        return forward_loss + inverse_loss

In [10]:
# Compute torques using inverse dynamics
def compute_torques(q, q_dot, q_ddot, mass_matrix_nn, potential_energy_nn):
    H = mass_matrix_nn(q)  # Mass matrix
    V = potential_energy_nn(q)  # Potential energy

    # Compute gradients
    V_grad = autograd.grad(V.sum(), q, create_graph=True)[0]

    H_grad = torch.stack([autograd.grad(H[i].sum(), q, create_graph=True, retain_graph=True)[0] for i in range(H.size(0))])

    print(H_grad.shape)
    H_dot_q = torch.einsum("bij,bj->bi", H_grad, q_dot)

    q_dot_H_grad_q_dot = torch.einsum("bi,bj,bij->b", q_dot, q_dot, H_grad) / 2.0

    # Compute torques
    tau = torch.bmm(H, q_ddot.unsqueeze(-1)).squeeze(-1) + H_dot_q + q_dot_H_grad_q_dot.unsqueeze(-1).squeeze(-1) + V_grad
    return tau

# Training code
def train_model(mass_matrix_nn, potential_energy_nn, dataloader, num_epochs=100, lr=1e-3):
    model_params = list(mass_matrix_nn.parameters()) + list(potential_energy_nn.parameters())
    optimizer = optim.Adam(model_params, lr=lr)
    criterion = DynamicsLoss()

    for epoch in range(num_epochs):
        epoch_loss = 0.0
        for batch in dataloader:
            q, q_dot, q_ddot = batch

            # Compute torques using inverse dynamics
            tau = compute_torques(q, q_dot, q_ddot, mass_matrix_nn, potential_energy_nn)

            optimizer.zero_grad()
            loss = criterion(q, q_dot, q_ddot, tau, mass_matrix_nn, potential_energy_nn)
            loss.backward()
            optimizer.step()

            epoch_loss += loss.item()

        print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {epoch_loss:.4f}")



In [11]:
# Example dataset setup

# Initialize neural networks
input_dim = q.size(1)  # Number of generalized coordinates
hidden_dim = 64
output_dim = input_dim
mass_matrix_nn = MassMatrixNN(input_dim, hidden_dim, output_dim)
potential_energy_nn = PotentialEnergyNN(input_dim, hidden_dim)

# Create dataloader
dataset = TensorDataset(q, q_dot, q_ddot)
dataloader = DataLoader(dataset, batch_size=32, shuffle=True)

# Train the model
train_model(mass_matrix_nn, potential_energy_nn, dataloader)

torch.Size([32, 32, 38])


ValueError: Size of label 'i' for operand 2 (38) does not match previous terms (32).